In [ ]:
# 練習 9-1
import cv2
import numpy as np

arr = np.array([
    [197, 25, 106, 156, 159],
    [149, 40, 107, 5, 71],
    [163, 198, 226, 223, 156],
    [222, 37, 68, 193, 157],
    [42, 72, 250, 41, 75]
], dtype=np.uint8)

h, w = arr.shape

padded = cv2.copyMakeBorder(arr, 1, 1, 1, 1, cv2.BORDER_REFLECT101)
# padded = np.pad(arr, pad_width=1, mode='reflect')

manual = np.zeros_like(arr)
# print(padded)

# 手動 3x3 均值濾波
for i in range(h):
    for j in range(w):
        window = padded[i:i+3, j:j+3]
        avg = int(np.sum(window) / 9)
        # manual[i][j] = avg
        manual[i][j] = np.round(np.sum(window) / 9).astype(np.uint8)

kernel = np.ones((3, 3), np.float32) / 9
opencv_result = cv2.filter2D(arr, -1, kernel)
Result = cv2.blur(arr,(3,3))

print("手動：\n", manual)
# print("OpenCV：\n", opencv_result)
print("OpenCV2：\n", Result)
print("是否完全一致？", np.array_equal(manual, Result))


In [ ]:
# 練習 9-2 均值濾波-real case
import cv2
import numpy as np

img=cv2.imread("C:/img/lenaNoise.png")


Result = cv2.blur(img,(3,3))
cv2.imshow("img", img)
cv2.imshow("cv2", Result)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# 練習 9-3
import cv2
import numpy as np

arr = np.array([
    [197, 25, 106, 156, 159],
    [149, 40, 107, 5, 71],
    [163, 198, 226, 223, 156],
    [222, 37, 68, 193, 157],
    [42, 72, 250, 41, 75]
], dtype=np.uint8)

ex = np.array([
    [1,2,1],
    [2,4,2],
    [1,2,1]
], dtype=np.int32)

h, w = arr.shape

padded = cv2.copyMakeBorder(arr, 1, 1, 1, 1, cv2.BORDER_REFLECT101)

manual = np.zeros_like(arr)

# 手動 3x3 均值濾波
for i in range(h):
    for j in range(w):
        window = padded[i:i+3, j:j+3]
        avg = int(np.sum(window) / 9)
        # manual[i][j] = np.around(np.sum(window * ex) / np.sum(ex)).astype(np.uint8) 0.5 會捨入到最近的偶數（banker’s rounding）
        manual[i][j] = np.floor(np.sum(window * ex) / np.sum(ex) + 0.5).astype(np.uint8)


Result = cv2.GaussianBlur(arr,(3,3),0)

print("手動：\n", manual)
print("OpenCV：\n", Result)
print("是否完全一致？", np.array_equal(manual, Result))


In [ ]:
# 練習 9-4 高斯濾波- real case
import cv2
import numpy as np

img=cv2.imread("C:/img/lenaNoise.png")


Result = cv2.GaussianBlur(img,(3,3),0)
cv2.imshow("img", img)
cv2.imshow("cv2", Result)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# 練習9-5
import cv2
import numpy as np

arr = np.array([
    [197, 25, 106, 156, 159],
    [149, 40, 107, 5, 71],
    [163, 198, 226, 223, 156],
    [222, 37, 68, 193, 157],
    [42, 72, 250, 41, 75]
], dtype=np.uint8)

h, w = arr.shape


padded = cv2.copyMakeBorder(arr, 1, 1, 1, 1, cv2.BORDER_REPLICATE)

manual = np.zeros_like(arr)

# 手動 3x3 中值濾波
for i in range(h):
    for j in range(w):
        window = padded[i:i+3, j:j+3]
        manual[i][j] = np.median(window).astype(np.uint8)

# OpenCV 中值濾波
Result = cv2.medianBlur(arr, 3)

print("手動：\n", manual)
print("OpenCV：\n", Result)
print("是否完全一致？", np.array_equal(manual, Result))


In [23]:
# 練習9-6 中值濾波-real case
import cv2
import numpy as np

img=cv2.imread("C:/img/lenaNoise.png")


Result = cv2.medianBlur(img,3)
cv2.imshow("img", img)
cv2.imshow("cv2", Result)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
# 練習9-7 雙邊濾波
import cv2
import numpy as np

img=cv2.imread("C:/img/bilTest.bmp")


# Result = cv2.bilateralFilter(img,300,100,100)

Result = cv2.bilateralFilter(img, d=9, sigmaColor=75, sigmaSpace=75)
Result2 = cv2.GaussianBlur(img,(111,111),0)
cv2.imshow("img", img)
cv2.imshow("bilateralFilter", Result)
cv2.imshow("GaussianBlur", Result2)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [48]:
# 練習9-8 綜合運用
import cv2
import numpy as np

# -----------------------------
# Step 0: 讀入影像 (灰階)
# -----------------------------
img = cv2.imread('C:/img/001.jpg', cv2.IMREAD_GRAYSCALE)

# Step 1: 濾波
# 使用 GaussianBlur 或中值濾波
blur = cv2.GaussianBlur(img, (5,5), 0)
# blur = cv2.medianBlur(img, 5)

# Step 2: 二值化
# 自動閾值或固定閾值
_, binary = cv2.threshold(blur, 90, 255, cv2.THRESH_BINARY)
# 或使用自適應閾值
# binary = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
#                                cv2.THRESH_BINARY, 11, 2)

# Step 3: 形態學操作
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
kernel2 = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
# 開運算 (去除小白點)
opening = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)

# 閉運算 (填補小黑洞)
closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)

# 膨脹與腐蝕
dilated = cv2.dilate(closing, kernel, iterations=1)
eroded = cv2.erode(dilated, kernel, iterations=1)

# Step 4: 連通元件分析
num_labels, labels = cv2.connectedComponents(eroded)

# Step 5: 區域上色
# 創建彩色影像來標記不同區域
height, width = labels.shape
color_map = np.zeros((height, width, 3), dtype=np.uint8)

# 隨機顏色
np.random.seed(42)
colors = [np.random.randint(0,255,3) for i in range(num_labels)]

for y in range(height):
    for x in range(width):
        label = labels[y,x]
        if label != 0:  # 0 是背景
            color_map[y,x] = colors[label]

# 顯示結果
cv2.imshow('Original', img)
cv2.imshow('Binary', binary)
cv2.imshow('Morphology', eroded)
cv2.imshow('Labeled', color_map)
cv2.waitKey(0)
cv2.destroyAllWindows()
